In [10]:
# Import pandas
import pandas as pd
# Import the required dependencies from sklearn
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
# Set the column width to view the text message data.
pd.set_option('max_colwidth', 200)
# Import Gradio
import gradio as gr

In [11]:
!pip install gradio


In [12]:
def sms_classification(sms_text_df):
    """
    Perform SMS classification using a pipeline with TF-IDF vectorization and Linear Support Vector Classification.

    Parameters:
    - sms_text_df (pd.DataFrame): DataFrame containing 'text_message' and 'label' columns for SMS classification.

    Returns:
    - text_clf (Pipeline): Fitted pipeline model for SMS classification.

    This function takes a DataFrame with 'text_message' and 'label' columns, splits the data into
    training and testing sets, builds a pipeline with TF-IDF vectorization and Linear Support Vector
    Classification, and fits the model to the training data. 
    The fitted pipeline is returned to make future predictions.
    """
    """
    Perform SMS classification using a pipeline with TF-IDF vectorization and Linear Support Vector Classification.
    """
    # Set the features variable to the text message column.
    X = sms_text_df['text_message']  
    # Set the target variable to the "label" column.
    y = sms_text_df['label']
    
    # Split data into training and testing and set the test_size = 33%
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    # Build a pipeline to transform the test set to compare to the training set.
    text_clf = Pipeline([
        ('tfidf', TfidfVectorizer(stop_words='english')),
        ('clf', LinearSVC()),
    ])
    
    # Fit the model to the transformed training data and return model.
    text_clf.fit(X_train, y_train)  
    print('Train Accuracy: %.3f' % text_clf.score(X_train, y_train))
    print('Test Accuracy: %.3f' % text_clf.score(X_test, y_test))
    return text_clf

In [13]:
# Load the dataset into a DataFrame
sms_text_df = pd.read_csv('Resources/SMSSpamCollection.csv')


In [14]:
# Call the sms_classification function with the DataFrame and set the result to the "text_clf" variable
text_clf = sms_classification(sms_text_df)


Train Accuracy: 1.000
Test Accuracy: 0.989


c:\Users\billy\anaconda3\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [15]:
# Create a function called `sms_prediction` that takes in the SMS text and predicts the whether the text is "not spam" or "spam". 
# The function should return the SMS message, and say whether the text is "not spam" or "spam".
def sms_prediction(text):
    """
    Predict the spam/ham classification of a given text message using a pre-trained model.

    Parameters:
    - text (str): The text message to be classified.

    Returns:
    - str: A message indicating whether the text message is classified as spam or not.

    This function takes a text message and a pre-trained pipeline model, then predicts the
    spam/ham classification of the text. The result is a message stating whether the text is
    classified as spam or not.
    """
    prediction = text_clf.predict([text])[0]
    if prediction == "ham":
        return f'The text message: "{text}", is not spam.'
    else:
        return f'The text message: "{text}", is spam.'

In [16]:
# Create a sms_app that takes a textbox for the inputs and has a textbox for the output.  
# Povide labels for each textbox. 
sms_app = gr.Interface(fn=sms_prediction, 
                       inputs=gr.Textbox(label="Enter SMS Text"), 
                       outputs=gr.Textbox(label="Prediction"))

    
# Launch the app.
sms_app.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Test the following text messages. 

---

1. You are a lucky winner of $5000! 
The text message: "You are a lucky winner of $5000!", is not spam.
2. You won 2 free tickets to the Super Bowl.
The text message: "You won 2 free tickets to the Super Bowl.", is not spam.
3. You won 2 free tickets to the Super Bowl text us to claim your prize.
The text message: "You won 2 free tickets to the Super Bowl text us to claim your prize.", is spam.
4. Thanks for registering. Text 4343 to receive free updates on medicare.
The text message: "Thanks for registering. Text 4343 to receive free updates on medicare.", is spam.